# L1 Reading要求：
* DLwP: C1&2 （已完成）
* <font color='red'>未完成</font> DL: C1 
* <font color='red'>未完成</font> Python DS Handbook



# DLwP C1
## 1.1
**Machine learning**: find useful representations of input data. 各种变形，比如linear projections, translations, nonlinear operations, etc.

** Deep Learning**, 又名layered representations learning, hierarchical representations learning.
* a multistage way to learn data representations
* depth = \# layers in the model
* 每一层的weights决定了transformation 如何参数化。我们需要找到合适的weights使得input map correctly to targets.
* 衡量方式：loss function/ objective function, compute distance between prediction & true value. 得到*loss score*.
* Optimizer: 通过backpropogation 去调整weights, 从而降低loss score.






## 1.2. Brief history of ML
1. Probabilistic Modeling. 
Eg. Naive Bayes, Logistic regression.

2. Kernel Methods.
Eg: SVM

3. Decision Tree. Random Forest. Gradient Boosting Machines-> 最好的适合nonperceptual data的算法

DL优点：不用烦feature engineering.







# DLwP C2

## 2.1. MNIST example


### Step 1. Load data

In [0]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Using TensorFlow backend.


11493376/11490434 [==============================] - 1s 0us/step


In [0]:
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)


(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


train set 有60000 samples, test set 有10000 samples. 
每个图像为 28x28 的 NumPy 数组，像素值介于 0 到 255 之间。标签是整数数组，介于 0 到 9 之间。


### Step 2. Network

In [0]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28*28,)))
network.add(layers.Dense(10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


**Layer**: Data processing module, 也可视为filter for data. <br>
Layer可以提取输入的data中的representation。
通常deep learning都是由一堆简单的layer堆叠而成，成为data distillation.

在上面的命令中，network由两层Dense layer组成, 是densely connected (又称为fully connected) neural layers。
第二层是10-way softmax layer, 会输出10个probability scores (sum=1)。每个score表示图像是属于对应的digit的概率。


### Step 3. Compilation 
* **Loss Function**: 衡量network在training data上的performance。并可以据此steer itself in the right direction
* **Optimizer**: 据此network update itself
* **Metrics to monitor during training and testing**: 这里我们只关心accuracy (fraction = #correctly classified/#all).



In [0]:
network.compile(optimizer='rmsprop',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

### Step 4. Preparing the image data
我们知道，输入的training images，shape是(60000, 28, 28)。每个像素值在[0, 255]之间，类型是uint8。但我们想把它们变成[0, 1]之间的float 32。所以：

In [0]:
train_images = train_images.reshape((60000, 28*28))
train_images = train_images.astype('float32')/255
test_images = test_images.reshape((10000, 28*28))
test_images = test_images.astype('float32')/255

<font color='red'>其实在老师给的例子里也没有这么麻烦，直接train_images=train_images/255.0，也没有把labels变成categorical。但这里不行，可能是因为那里直接从tf.keras调用？看看后面再说</font>

解决了，老师给的例子里是先除以255.0，就变成了float; 然后flatten,就是reshape了。 

In [0]:
from keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

### Step 5. Fit data to model

In [0]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
60000/60000 [==============================] - 6s 98us/step - loss: 0.2530 - acc: 0.9269
Epoch 2/5
60000/60000 [==============================] - 5s 86us/step - loss: 0.1024 - acc: 0.9694
Epoch 3/5
60000/60000 [==============================] - 5s 86us/step - loss: 0.0676 - acc: 0.9797
Epoch 4/5
60000/60000 [==============================] - 5s 86us/step - loss: 0.0492 - acc: 0.9855
Epoch 5/5
60000/60000 [==============================] - 5s 88us/step - loss: 0.0377 - acc: 0.9882


上面给出了training中的两个数值： **loss** & **accuracy**.

### Step 6. Test fitted model

In [0]:
test_loss, test_acc = network.evaluate(test_images, test_labels)
print(test_loss, test_acc)

10000/10000 [==============================] - 1s 53us/step
0.06998087865991984 0.9781


## 2.2. Data representations for neural networks

1. 讲了tensor的定义，ndim等等。

2. batch。把数据分成一部分一部分的去train. 按第一个轴（对应sample 编号）划分，所以first axis (axis 0) is called the batch axis or batch dimension

3. 常见的data tensors:
> * Vector data: 2D tensors (samples, features)
> * Timeseries data (sequence data): 3D tensors (samples, timesteps, features)
> * Images: 4D tensors (samples, height, width, channels)或者(samples, channels, height, width)
> * Video: 5D tensors (samples, frames, height, width, channels) 或者(samples, frames, channels, height, width)









## 2.3. Tensor operations

1. element-wise operation

2. Broadcasting. 不同shape的tensors间的操作，输出的ndim是输入中ndim更大的那个。<br>
 Broadcasting consists of two steps: <br>
1) Axes (called broadcast axes) are added to the smaller tensor to match the ndim of the larger tensor.<br>
2) The smaller tensor is repeated alongside these new axes to match the full shape of the larger tensor.

3. Tensor dot, 也称为tensor product. 就是点乘，比如矢量间点乘为标量；矩阵点乘矢量为矢量。

4. Tensor reshaping


## 2.4. Gradient-based Optimization

### 1. Training loop
随机、小值初始化weights (kernal&bias), 通过forward, 用feedback signal 逐渐调整weights:
* 提取training x & y batch.
* 在x上run network (forward pass)， 得到y_pred.
* 比较network loss, 即y与y_pred间差异
* 更新网络所有weights, 从而减小这个batch的loss.

最后一步的做法：考虑到网络中所有操作都是可微分的，计算**gradient** of the loss with regard to the network's coefiicients.

### 2. 一些取值：
* Step factor: <br>
不能太小，否则可能stuck in a local minimum. 不能太大， 否则可能end up taking you to completely random locations on the curve.

* Batch size: <br>
在特别小的mini-batch SGD 和整体的batch SGD中找到平衡。

### 3. Optimizers: 
比如Adam, RMSProp, SGD....
带momentum的GD就不担心掉到local minimum里。

Momentum is implemented by moving the ball at each step based not only on the current slope value (current acceleration) but also on the current velocity (resulting from past acceleration). In practice, this means updating the parameter w based not only on the current gradient value but also on the previous parameter update.

### 4. Backpropagation
又称为reverse-mode differentiation.
用到chain rule



## 2.5.
Epoch: each iteration over all the training data is called an epoch

对于开始的例子，60000个samples, 每个batch包括128个sample, 总共5个epoch. 所以参数更新了5x(60000/128) = 2345次。


## Chapter Summary:
* **Learning** means finding a combination of model parameters that minimizes a loss function for a given set of training data samples and their corresponding targets.
* Learning happens by drawing random batches of data samples and their targets, and computing the gradient of the network parameters with respect to the loss on the batch. The network parameters are then moved a bit (the magnitude of the move is defined by the learning rate) in the opposite direction from the gradient.
* The entire learning process is made possible by the fact that neural networks are chains of differentiable tensor operations, and thus it’s possible to apply the chain rule of derivation to find the gradient function map- ping the current parameters and current batch of data to a gradient value.
* Two key concepts you’ll see frequently in future chapters are **loss and optimizers**. These are the two things you need to define before you begin feeding data into a network.
* The **loss** is the quantity you’ll attempt to minimize during training, so it should represent a measure of success for the task you’re trying to solve.
* The optimizer specifies the exact way in which the gradient of the loss will be used to update parameters: for instance, it could be the RMSProp optimizer, SGD with momentum, and so on.